# Imports

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import pandas as pd
import numpy as np

In [9]:
pd.set_option('display.max_columns', 25)

In [10]:
import sys
# sys.path.append(r'D:\Usuarios\Kitty\Documentos\1-Master\TFM')
sys.path.append(r'D:\Programas\GitHub\TFM')

# Load Data

In [33]:
df = pd.read_csv("Credit_Card.csv").drop(columns=["ID"])

In [34]:
df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000.0,1,3,1,39,0,0,0,0,0,0,188948.0,192815.0,208365.0,88004.0,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0
29996,150000.0,1,3,2,43,-1,-1,-1,-1,0,0,1683.0,1828.0,3502.0,8979.0,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0
29997,30000.0,1,2,2,37,4,3,2,-1,0,0,3565.0,3356.0,2758.0,20878.0,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1
29998,80000.0,1,3,1,41,1,-1,0,0,0,-1,-1645.0,78379.0,76304.0,52774.0,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1


# Parameters

In [41]:
target = "Default"

# Eliminamos categorías outliers

In [ ]:
# Identificar columnas categóricas automáticamente
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# Iterar sobre cada columna categórica
for col in categorical_columns:
    # Calcular la frecuencia relativa de cada categoría
    frecuencia_relativa = df[col].value_counts(normalize=True)
    
    # Identificar las categorías que representan menos del 1%
    categorias_validas = frecuencia_relativa[frecuencia_relativa >= 0.01].index
    
    # Filtrar el DataFrame para conservar solo las categorías válidas
    df = df[df[col].isin(categorias_validas)]

# Mostrar el DataFrame resultante
print("Dataset después de eliminar registros con categorías raras (<1%) en todas las columnas categóricas:")
print(df)


# One-Hot Encoder

In [ ]:
# Identificar columnas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# Realizar One-Hot Encoding para todas las columnas categóricas
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=False)

# Mostrar el DataFrame resultante
print("Dataset después del One-Hot Encoding:")
print(df_encoded)

# Scale & Train

In [ ]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

# 1. Cargar el dataset
X = pd.DataFrame(df_encoded[[elem for elem in df_encoded.columns if elem is not target]], columns=df_encoded.columns)
y = pd.Series(df_encoded[target], name=target)

# 2. Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. Configurar validación cruzada k-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 4. Inicializar modelos
models = {
    "Regresión Logística": LogisticRegression(max_iter=1000, random_state=42),
    "Árbol de Decisión": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# 5. Configurar los hiperparámetros para cada modelo
param_grids = {
    "Regresión Logística": {
        "C": [0.1, 1, 10, 100],
        "solver": ["lbfgs", "liblinear"],
        "penalty": ["l2"]
    },
    "Árbol de Decisión": {
        "criterion": ["gini", "entropy"],
        "max_depth": [3, 5, 10, None],
        "min_samples_split": [2, 5, 10]
    },
    "Naive Bayes": {
        "var_smoothing": np.logspace(0, -9, num=10)
    },
    "K-Nearest Neighbors": {
        "n_neighbors": [3, 5, 7, 9],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan"]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 10, None],
        "min_samples_split": [2, 5, 10]
    },
    "Gradient Boosting": {
        "n_estimators": [50, 100, 200],
        "learning_rate": [0.01, 0.1, 0.2],
        "max_depth": [3, 5, 10]
    }
}

# 6. Diccionario para almacenar los mejores modelos y resultados
best_models = {}
grid_results = []

# 7. Iterar sobre cada modelo y realizar GridSearchCV
for model_name, model in models.items():
    grid = GridSearchCV(
        estimator=model,
        param_grid=param_grids[model_name],
        cv=kfold,
        scoring="accuracy",
        n_jobs=-1
    )
    grid.fit(X_scaled, y)
    best_models[model_name] = grid.best_estimator_
    grid_results.append({
        "Model": model_name,
        "Best Params": grid.best_params_,
        "Best Accuracy": grid.best_score_
    })

# Convertir resultados de hiperparámetros en un DataFrame
grid_results_df = pd.DataFrame(grid_results)
grid_results_df.sort_values(by="Best Accuracy", ascending=False, inplace=True)

# 8. Evaluar los mejores modelos con k-fold
evaluation_summary = []

for model_name, best_model in best_models.items():
    accuracy = cross_val_score(best_model, X_scaled, y, cv=kfold, scoring="accuracy").mean()
    precision = cross_val_score(best_model, X_scaled, y, cv=kfold, scoring="precision_weighted").mean()
    recall = cross_val_score(best_model, X_scaled, y, cv=kfold, scoring="recall_weighted").mean()
    f1 = cross_val_score(best_model, X_scaled, y, cv=kfold, scoring="f1_weighted").mean()
    
    evaluation_summary.append({
        "Model": model_name,
        "Mean Accuracy": accuracy,
        "Mean Precision": precision,
        "Mean Recall": recall,
        "Mean F1-Score": f1
    })

# Convertir resultados de evaluación en un DataFrame
evaluation_df = pd.DataFrame(evaluation_summary)
evaluation_df.sort_values(by="Mean Accuracy", ascending=False, inplace=True)

# 9. Mostrar resultados finales
print("Resultados de búsqueda de hiperparámetros:")
print(grid_results_df)

print("\nDesempeño de los modelos optimizados:")
print(evaluation_df)


d:\miniconda3\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
d:\miniconda3\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
d:\miniconda3\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


ValueError: 
All the 80 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "d:\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\miniconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1223, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "d:\miniconda3\Lib\site-packages\sklearn\base.py", line 650, in _validate_data
    X, y = check_X_y(X, y, **check_params)
           ^^^^^^^^^^